In [13]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import torch.nn.init as init
from sklearn.linear_model import LinearRegression

def regression(df:pd.DataFrame,window,time):
    df = df[df['Close'].notna()]
    df = df[df['metrics'].notna()]


    tensor_list = [df['metrics'].values[i:i+window] for i in range(len(df) - window-time)]
    X = np.array(tensor_list)
    # X = torch.from_numpy(X)

    tensor_list = [df['Close'].values[i:i+1] for i in range(window,len(df)-time)]
    Y = np.array(tensor_list)
    # Y = torch.from_numpy(Y)


    tensor_list = [df['metrics'].values[i:i+window] for i in range(len(df) - window-time,len(df) - window)]
    X_test = np.array(tensor_list)
    # X_test = torch.from_numpy(X_test)

    tensor_list = [df['Close'].values[i:i+1] for i in range(len(df)-time,len(df))]
    Y_test = np.array(tensor_list)
    # Y_test = torch.from_numpy(Y_test)


    print(len(X),len(Y))
    print(len(X_test),len(Y_test))

    # 创建线性回归模型
    model = LinearRegression()

    # 拟合模型
    model.fit(X, Y)

    # 打印模型系数
    print('模型系数:', model.coef_)  # 斜率
    print('截距:', model.intercept_)  # 截距




    predicted_y = model.predict(X)
    # print(predicted_y)


    print(np.corrcoef(Y.ravel(), predicted_y.ravel())[0, 1])




    # # 定义线性回归模型
    # class LinearRegression(nn.Module):
    #     def __init__(self, input_size):
    #         super(LinearRegression, self).__init__()
    #         self.linear = nn.Linear(input_size, 1,dtype=torch.float64,bias=False)
    #         # init.normal_(self.linear.weight, mean=0, std=0.01)
    #         # init.zeros_(self.linear.weight)


    #     def forward(self, x):
    #         return self.linear(x)

    # # 初始化模型
    # model = LinearRegression(window)
    # # 定义损失函数
    # criterion = nn.MSELoss()
    # # 定义优化器
    # optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)


    # Y_t = Y.numpy().ravel()
    # Y_test_t = Y_test.numpy().ravel()


    # # 训练模型
    # epochs = 50000
    # r2_max = 0
    # corr = 0
    # return_str = ''
    # corrs = [[],[]]
    # for epoch in range(epochs):
    #     model.train()

        

    #     # 前向传播
    #     y_pred = model(X)

    #     # 计算损失
    #     loss = criterion(y_pred, Y)
    #     # 反向传播
    #     optimizer.zero_grad()
    #     loss.backward()
    #     optimizer.step()
    #     if (epoch+1) % 1 == 0:
            
    #         y_pred_t = y_pred.detach().numpy().ravel()
    #         correlation_coefficient = np.corrcoef(Y_t, y_pred_t)[0, 1]
    #         r2 = r2_score(Y.numpy().ravel(), y_pred.detach().numpy().ravel())

    #         model.eval()
    #         y_test_pred = model(X_test)
    #         y_test_pred_t = y_test_pred.detach().numpy().ravel()
    #         correlation_coefficient_test = np.corrcoef(Y_test_t, y_test_pred_t)[0, 1]
    #         corrs[0].append(correlation_coefficient_test)
    #         corrs[1].append(epoch+1)

    #         # correlation_coefficient_test=0
    #         if correlation_coefficient > corr:
    #             corr = correlation_coefficient
    #             return_str = '{:.4f};{:.4f};{:.4f}'.format(loss.item(), r2, correlation_coefficient)
    #         if r2 > r2_max:
    #             r2_max =r2
    #             return_str = '{:.4f};{:.4f};{:.4f}'.format(loss.item(), r2, correlation_coefficient)
            
            
    #         print('Epoch [{}/{}], Loss: {:.4f}, R2 Score: {:.4f}, train: {:.4f},Corr Score: {:.4f}'.format(epoch+1, epochs, loss.item(), r2, correlation_coefficient,correlation_coefficient_test))
    #         # print(model.linear.weight)
    # # print(model.linear.weight)
    # return return_str,model.linear.weight.detach().numpy().ravel(),corrs



scaler = MinMaxScaler()
df = pd.read_csv('sentiment_all_feature.csv')
# fear = pd.read_csv('Fear_and_Greed_Index.csv')
# df = pd.merge(df,fear,left_on='date',right_on='timestamp',how='left')
df['metrics'] = df['metrics']
print(df)


df['Close'] = df['Close'].pct_change()





for window in [7]:
    for diff_type in ['pct_change']:
        if diff_type == 'pct_change':
            df[['metrics']]= scaler.fit_transform(df[['metrics']])
            df = df[df['metrics']!=0]
            # print(df[df['metrics']==0])
            df['metrics'] = df['metrics'].pct_change()

        elif diff_type == 'diff':
            df['metrics'] = df['metrics'].diff()
        elif diff_type == 'log_return':
            df[['metrics']]= scaler.fit_transform(df[['metrics']])
            
            df = df[df['metrics']!=0]
            df['metrics'] = np.log(df['metrics']) - np.log(df['metrics'].shift(1))
        else:
            df['metrics'] = df['metrics']

        # 加入发推量
        # df[['count']]= scaler.fit_transform(df[['count']])
        # df['metrics'] = df['metrics'] * df['count']


        regression(df, window,0)
        # res,param,corrs = regression(df, window,0)
        # import matplotlib.pyplot as plt
        # plt.plot(corrs[1], corrs[0],label='Corr')
        # plt.xlabel('Epoth')
        # plt.ylabel('Test Corr')
        # plt.legend()
        # plt.show()
        # print("{};{};{};{}".format(window, diff_type, res, param))

            date       metrics         Close  count
0     2020-01-01 -1.311437e+18   7200.174316   8317
1     2020-01-02  1.154228e+13   6985.470215   9861
2     2020-01-03 -1.708432e+18   7344.884277  10988
3     2020-01-04 -1.145684e+16   7410.656738   9404
4     2020-01-05 -7.966623e+23   7411.317383   9094
...          ...           ...           ...    ...
1242  2023-05-27 -2.412345e+18  26868.353516  12921
1243  2023-05-28  3.793781e+17  28085.646484  13341
1244  2023-05-29  8.398361e+15  27745.884766  15120
1245  2023-05-30 -2.825257e+18  27702.349609  17600
1246  2023-05-31  1.731816e+17  27219.658203  17008

[1247 rows x 4 columns]
1238 1238
0 0
模型系数: [[ 0.00168244 -0.0073721  -0.00376362 -0.00164101  0.00329096 -0.00582283
   0.00409029]]
截距: [0.00167102]
0.026649741158220757


In [1]:
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
# 读取CSV文件
data = pd.read_csv("sentiment.csv")

# 提取"Close"和"metrics"列数据


def test(lags,data):
    price = data["Close"]
    metrics = data["metrics"]
    # lags = 7
    price = price[lags:]

    # 创建滞后外生变量的数据集
    lagged_metrics = pd.DataFrame()
    for lag in range(1, lags+1):
        lagged_metrics[f"metrics_lag{lag}"] = metrics.shift(lag)

    # 将滞后外生变量和原始数据合并
    exog_data = pd.concat([lagged_metrics], axis=1)
    exog_data = exog_data[exog_data[f'metrics_lag{lags}'].notna()]

    # print(exog_data)
    # print(price)


    time = 500
    train_data = price[:-time]
    test_data = price[-time:]
    train_sentiment = exog_data[:-time]
    test_sentiment = exog_data[-time:]


    # print(len(train_data),len(train_sentiment))
    # print(len(test_data),len(test_sentiment))

    # print(test_data.values)
    # 创建并拟合AutoReg模型
    add_sentiment = 0
    if not add_sentiment:
        model = AutoReg(train_data.values, lags=lags)
    else:
        model = AutoReg(train_data.values, lags=lags,exog=train_sentiment.values)
    model_fit = model.fit()


    # 预测未来100个时间点的股价
    if add_sentiment:
        predictions = model_fit.predict(start=len(train_data), end=len(train_data)+len(test_data)-1,exog_oos=test_sentiment.values)
    else:
        predictions = model_fit.predict(start=len(train_data), end=len(train_data)+len(test_data)-1)
    # 计算预测结果的均方误差
    # print(predictions.values)
    mse = mean_squared_error(test_data, predictions)
    print('Mean Squared Error:', mse)
    # 计算R2
    r2 = r2_score(test_data, predictions)
    print('R2 Score:', r2)
    # 计算Corr
    correlation = np.corrcoef(test_data, predictions)[0, 1]
    print(correlation)
    # 绘制预测结果

    # plt.plot(train_data.index, train_data.values, label='Train Data')
    plt.plot(test_data.index, test_data.values, label='Test Data')
    plt.plot(test_data.index, predictions, label='Predictions')
    plt.legend()
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title('Stock Price Prediction')
    # plt.show()


    add_sentiment = 1
    if not add_sentiment:
        model = AutoReg(train_data.values, lags=lags)
    else:
        model = AutoReg(train_data.values, lags=lags,exog=train_sentiment.values)
    model_fit = model.fit()


    # 预测未来100个时间点的股价
    if add_sentiment:
        predictions = model_fit.predict(start=len(train_data), end=len(train_data)+len(test_data)-1,exog_oos=test_sentiment.values)
    else:
        predictions = model_fit.predict(start=len(train_data), end=len(train_data)+len(test_data)-1)
    # 计算预测结果的均方误差
    # print(predictions.values)
    mse = mean_squared_error(test_data, predictions)
    print('Mean Squared Error:', mse)
    # 计算R2
    r2 = r2_score(test_data, predictions)
    print('R2 Score:', r2)
    # 计算Corr
    correlation = np.corrcoef(test_data, predictions)[0, 1]
    print(correlation)
    # 绘制预测结果
    # plt.plot(train_data.index, train_data.values, label='Train Data')
    # plt.plot(test_data.index, test_data.values, label='Test Data')
    plt.plot(test_data.index, predictions, label='Predictions')
    plt.legend()
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title('BTC Price Prediction')
    # plt.show()
test(7,data.copy())

Mean Squared Error: 345389816.8179955


NameError: name 'r2_score' is not defined

In [ ]:
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
# 读取CSV文件
data = pd.read_csv("sentiment_.csv")
data['Close'] = data['Close'].pct_change()



# 加入发推量
# data[['count']]= scaler.fit_transform(data[['count']])
# data['metrics'] = data['metrics'] * df['count']

data[['metrics']]= scaler.fit_transform(data[['metrics']])
data = data[data['metrics']!=0]
data['metrics'] = data['metrics'].pct_change()

data = data[data['Close'].notna()]
data = data[data['metrics'].notna()]
# 提取"Close"和"metrics"列数据
data = data.reset_index(drop=True)
# print(data)



def test(data,lags,time):
    price = data["Close"]
    metrics = data['metrics']



    # lags = 7
    price = price[lags:]

    # 创建滞后外生变量的数据集
    lagged_metrics = pd.DataFrame()
    for lag in range(1, lags+1):
        lagged_metrics[f"metrics_lag{lag}"] = metrics.shift(lag)

    # 将滞后外生变量和原始数据合并
    exog_data = pd.concat([lagged_metrics], axis=1)
    exog_data = exog_data[exog_data[f'metrics_lag{lags}'].notna()]


    # exog_data = exog_data[lags:]

    # print(exog_data)
    # print(price)


    # time = 800
    train_data = price[:-time]
    test_data = price[-time:]
    train_sentiment = exog_data[:-time]
    test_sentiment = exog_data[-time:]


    add_sentiment = 1
    # print(len(train_data),len(train_sentiment))
    # 创建并拟合AutoReg模型
    if add_sentiment:
        model = AutoReg(train_data.values, lags=lags, exog=train_sentiment.values)
    else:
        model = AutoReg(train_data.values, lags=lags)
    model_fit = model.fit()
    # print(model_fit.params)

    # 预测未来100个时间点的股价
    if add_sentiment:
        predictions = model_fit.predict(start=len(train_data), end=len(train_data)+len(test_data)-1,exog_oos=test_sentiment.values)
    else:
        predictions = model_fit.predict(start=len(train_data), end=len(train_data)+len(test_data)-1)
    # 计算预测结果的均方误差
    # print(predictions.values)
    mse = mean_squared_error(test_data, predictions)
    # print('Mean Squared Error:', mse)
    # 计算R2
    r2 = r2_score(test_data, predictions)
    # print('R2 Score:', r2)
    # 计算Corr
    correlation1 = np.corrcoef(test_data, predictions)[0, 1]
    # print('Corr:',correlation1)
    # 绘制预测结果
    # plt.plot(train_data.index, train_data.values, label='Train Data')
    # plt.plot(test_data.index, test_data.values, label='Test Data')
    # plt.plot(test_data.index, predictions, label='Predictions')
    # plt.legend()
    # plt.xlabel('Date')
    # plt.ylabel('Close Price')
    # plt.title('Stock Price Prediction')
    # plt.show()


    add_sentiment = 0
    # print(len(train_data),len(train_sentiment))
    # 创建并拟合AutoReg模型
    if add_sentiment:
        model = AutoReg(train_data.values, lags=lags, exog=train_sentiment.values)
    else:
        model = AutoReg(train_data.values, lags=lags)
    model_fit = model.fit()


    # 预测未来100个时间点的股价
    if add_sentiment:
        predictions = model_fit.predict(start=len(train_data), end=len(train_data)+len(test_data)-1,exog_oos=test_sentiment.values)
    else:
        predictions = model_fit.predict(start=len(train_data), end=len(train_data)+len(test_data)-1)
    # 计算预测结果的均方误差
    # print(predictions.values)
    mse = mean_squared_error(test_data, predictions)
    # print('Mean Squared Error:', mse)
    # 计算R2
    r2 = r2_score(test_data, predictions)
    # print('R2 Score:', r2)
    # 计算Corr
    correlation2 = np.corrcoef(test_data, predictions)[0, 1]
    # print('Corr:',correlation2)
    # 绘制预测结果
    # plt.plot(train_data.index, train_data.values, label='Train Data')
    # plt.plot(test_data.index, test_data.values, label='Test Data')
    # plt.plot(test_data.index, predictions, label='Predictions')
    # plt.legend()
    # plt.xlabel('Date')
    # plt.ylabel('Close Price')
    # plt.title('Stock Price Prediction')
    # plt.show()
    return correlation1,correlation2
# corr1,corr2 = test(data.copy(),7,100)
corr1_l = []
corr2_l = []
for i in range(100,1200,100):
    corr1,corr2 = test(data.copy(),7,i)
    corr1_l.append(corr1)
    corr2_l.append(corr2)
    print(corr1,corr2)
plt.plot(range(100,1200,100), corr1_l, label='add sentiment')
plt.plot(range(100,1200,100), corr2_l, label='origin')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.title('Stock Price Prediction')
plt.show()